## V8 是如何工作的

<img src='https://raw.githubusercontent.com/binghuis/assets/main/excalidraw/v8.avif' width='580px' />

JS 源码以 `UTF-16` 编码字符串的形式传给 V8 引擎。

**词法分析阶段**

V8 引擎扫描器扫描全量代码，进行词法分析，将源码分解成一系列标记（`token`）。

**延迟解析**

如果一段代码接下来不会被执行，那么这段代码仅被预解析，将来这段代码被执行时再解析，这就是「延迟解析」。

预解析会进行简单的语法检查，并对变量和函数进行位置记录，这些记录信息后续会被用于语法分析。

**语法分析阶段**

针对将被执行的代码，解析器会根据词法分析阶段生成的标记进行语法分析，生成 AST 抽象语法树。
语法分析阶段代码结构确定，此时作用域确定。

**解释执行**

语法分析阶段生成的 AST 抽象语法树被 JS 解释器编译成字节码并解释执行。

**JIT 即时编译**

如果一段代码被解释器执行多次则会被标记为热点代码，热点代码将会被 JS 编译器进一步优化编译成机器码并执行。这就是「JIT 即时编译技术」。

如果这段被优化编译的代码结构发生了变化，那么编译器将会对这段代码反优化编译成字节码，并交回解释器执行。


## 什么是执行上下文

JS 代码执行分两阶段：

- 创建阶段
- 执行阶段

在代码真正执行之前的创建阶段，V8 会创建一个执行上下文作为代码的执行环境。执行上下文有两个重要的状态组件：

- 词法环境（Lexical Environment）
- 变量环境（Variable Environment）

词法环境和变量环境都是基于 ES6 规范中的词法环境（LexicalEnvironment）结构的。这个结构包括：

- 环境记录（存变量和函数声明）
- 对外部词法环境的引用（可能为空）

这里需要注意的是，执行上下文中的词法环境和 ES6 规范中的词法环境是两个不同的概念。

- 执行上下文中的词法环境（LexicalEnvironment）是一个状态组件，用于跟踪标识符（变量或函数）的状态。
- ES6 规范中的词法环境（Lexical Environment）是一个结构，包含环境记录和对外部词法环境的引用。

执行上下文中的词法环境和变量环境组件都是基于词法环境结构的。

在上下文初始化阶段，var 变量和函数声明被添加到变量环境的环境记录，此时 var 变量和函数声明都被分配了内存空间，var 变量初始化值为 undefined。由于在创建阶段 var 和函数就已完成了初始化，因此在它们声明之前就能被调用或执行，这就是「提升」。
初始化阶段词法环境指向变量环境，它们的环境记录和对外部词法环境的引用都相同。

在代码真正的执行阶段，当遇到 `let`、`const` 声明的变量时，这些变量会被添加到词法环境的环境记录，此时变量被分配内存，但未被初始化，在变量被赋值之前是不能被访问的，这就是「暂时性死区」。


## 作用域和词法环境结构

作用域用于描述变量的可见性和生命周期。

当你在代码中创建一个新的作用域（比如通过一个函数声明或一个块语句），V8 就会创建一个新的词法环境，其环境记录包含新作用域中的变量和函数，对外部词法环境的引用则指向创建这个新作用域的词法环境。

这就形成了一个链式结构，也就是所谓的 "作用域链"。当你试图访问一个变量时，JavaScript 引擎会首先在当前的词法环境中查找，如果找不到，就会沿着对外部词法环境的引用去外部词法环境中查找，以此类推，直到找到变量或者到达全局作用域。如果在全局作用域中仍然找不到这个变量，JavaScript 就会抛出一个 `ReferenceError`。
